In [2]:
import pandas as pd
import pickle
import plotly.graph_objs as go
import latextable
from texttable import Texttable
from strategy.strategy import (
    BuyAndHoldStrategy,
    MACDStrategy,
    RSIStrategy,
    ModelQuantilePredictionsStrategy,
    ModelGmadlPredictionsStrategy,
    ConcatenatedStrategies
)
from strategy.util import (
    get_data_windows,
    get_sweep_window_predictions,
    get_predictions_dataframe
)
from strategy.evaluation import (
    parameter_sweep,
    evaluate_strategy
)
from strategy.plotting import (
    plot_sweep_results
)

PADDING=5000
VALID_PART=0.2
INTERVAL='5min'
METRIC='mod_ir'
TOP_N=10

In [3]:
data_windows = get_data_windows(
    'wne-masters-thesis-testing',
    'btc-usdt-5m:latest',
    min_window=0, 
    max_window=5
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Downloading large artifact btc-usdt-5m:latest, 745.12MB. 12 files... 
wandb:   12 of 12 files downloaded.  
Done. 0:0:1.3


In [3]:
def sweeps_on_all_windows(data_windows, strategy_class, params, **kwargs):
    result = []
    for in_sample, _ in data_windows:
        data_part = int((1 - VALID_PART) * len(in_sample))
        result.append(parameter_sweep(in_sample[data_part-PADDING:], strategy_class, params, padding=PADDING, interval=INTERVAL, **kwargs))
    return result

In [4]:
buyandhold_best_strategies = [BuyAndHoldStrategy() for _ in data_windows] 

In [5]:
MACD_PARAMS = {
    'fast_window_size': [2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584],
    'slow_window_size': [2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584],
    'signal_window_size': [2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584],
    'short_sell': [True, False]
}
MACD_PARAMS_FILTER = lambda p: (
    p['slow_window_size'] > p['fast_window_size']
)
macd_sweep_results = sweeps_on_all_windows(
    data_windows,
    MACDStrategy,
    MACD_PARAMS,
    params_filter=MACD_PARAMS_FILTER,
    sort_by=METRIC)
macd_best_strategies = [[strat for _, strat in results[:TOP_N]] for results in macd_sweep_results]

100%|██████████| 3840/3840 [00:18<00:00, 202.96it/s]


In [98]:
# plot_sweep_results(pd.DataFrame([result for result, _ in macd_sweep_results[0]]), parameters=MACD_PARAMS.keys(), objective=METRIC)
# macd_sweep_results[0][:3]
# macd_best_strategies

In [6]:
RSI_FILTER = lambda p: (
    ((p['enter_long'] is not None and (p['enter_short'] is not None or p['exit_long'] is not None))
    or (p['enter_short'] is not None and (p['exit_short'] is not None or p['enter_long'] is not None)))
    and (p['enter_short'] is None or p['exit_long'] is None or (p['exit_long'] > p['enter_short']))
    and (p['enter_long'] is None or p['exit_short'] is None or (p['exit_short'] < p['enter_long'])))

RSI_PARAMS = {
    'window_size': [2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584],
    'enter_long': [None, 70, 75, 80, 85, 90, 95],
    'exit_long': [None, 5, 10, 15, 20, 25, 30],
    'enter_short': [None, 5, 10, 15, 20, 25, 30],
    'exit_short': [None, 70, 75, 80, 85, 90, 95],
}
rsi_sweep_results = sweeps_on_all_windows(data_windows, RSIStrategy, RSI_PARAMS, params_filter=RSI_FILTER, sort_by=METRIC)
rsi_best_strategies = [[strat for _, strat in results[:TOP_N]] for results in rsi_sweep_results]

100%|██████████| 11088/11088 [00:52<00:00, 209.32it/s]


In [100]:
# plot_sweep_results(pd.DataFrame([result for result, _ in rsi_sweep_results[0]]), parameters=RSI_PARAMS.keys(), objective=METRIC)

In [8]:
# Model with rmse loss
SWEEP_ID = 'filipstefaniuk/wne-masters-thesis-testing/9afp99kz'
train_pred_windows = get_sweep_window_predictions(SWEEP_ID, 'train')
valid_pred_windows = get_sweep_window_predictions(SWEEP_ID, 'valid')
test_pred_windows = get_sweep_window_predictions(SWEEP_ID, 'test')

wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  


In [9]:
MODEL_RMSE_LOSS_FILTER = lambda p: (
    ((p['enter_long'] is not None and (p['enter_short'] is not None or p['exit_long'] is not None))
    or (p['enter_short'] is not None and (p['exit_short'] is not None or p['enter_long'] is not None)))
    and (p['enter_short'] is None or p['exit_long'] is None or (p['exit_long'] > p['enter_short']))
    and (p['enter_long'] is None or p['exit_short'] is None or (p['exit_short'] < p['enter_long'])))

rmse_model_sweep_results = []
for (in_sample, _), train_preds, valid_preds, test_preds in zip(data_windows, train_pred_windows, valid_pred_windows, test_pred_windows):
    data_part = int((1 - VALID_PART) * len(in_sample))
    params={
        'predictions': [get_predictions_dataframe(train_preds, valid_preds, test_preds)],
        'enter_long': [None, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007],
        'exit_long': [None, -0.001, -0.002, -0.003, -0.004, -0.005, -0.006, -0.007],
        'enter_short': [None, -0.001, -0.002, -0.003, -0.004, -0.005, -0.006, -0.007],
        'exit_short': [None, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007],
        # 'enter_short': [None],
        # 'exit_short': [None],
    }
    
    rmse_model_sweep_results.append(parameter_sweep(
        in_sample[data_part-PADDING:],
        ModelGmadlPredictionsStrategy,
        params,
        params_filter=MODEL_RMSE_LOSS_FILTER,
        padding=PADDING,
        interval=INTERVAL,
        sort_by=METRIC))
    

rmse_model_best_strategies = [[strat for _, strat in results[:TOP_N]] for results in rmse_model_sweep_results]

100%|██████████| 1176/1176 [02:12<00:00,  8.85it/s]


In [10]:
# Model with quantile loss
SWEEP_ID = 'filipstefaniuk/wne-masters-thesis-testing/8m3hwwmx'
train_pred_windows = get_sweep_window_predictions(SWEEP_ID, 'train')
valid_pred_windows = get_sweep_window_predictions(SWEEP_ID, 'valid')
test_pred_windows = get_sweep_window_predictions(SWEEP_ID, 'test')

wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  


In [11]:
MODEL_QUANTILE_LOSS_FILTER = lambda p: (
    ((p['quantile_enter_long'] is not None and (p['quantile_enter_short'] is not None or p['quantile_exit_long'] is not None))
    or (p['quantile_enter_short'] is not None and (p['quantile_exit_short'] is not None or p['quantile_enter_long'] is not None)))
    and (p['quantile_enter_short'] is None or p['quantile_exit_long'] is None or (p['quantile_exit_long'] < p['quantile_enter_short']))
    and (p['quantile_enter_long'] is None or p['quantile_exit_short'] is None or (p['quantile_exit_short'] < p['quantile_enter_long'])))

quantile_model_sweep_results = []
for (in_sample, _), train_preds, valid_preds, test_preds in zip(data_windows, train_pred_windows, valid_pred_windows, test_pred_windows):
    data_part = int((1 - VALID_PART) * len(in_sample))
    params={
        'predictions': [get_predictions_dataframe(train_preds, valid_preds, test_preds)],
        'quantiles': [[0.01, 0.02, 0.03, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.97, 0.98, 0.99]],
        'quantile_enter_long': [None, 0.9, 0.95, 0.97, 0.98, 0.99],
        'quantile_exit_long': [None, 0.9, 0.95, 0.97, 0.98, 0.99],
        'quantile_enter_short': [None, 0.9, 0.95, 0.97, 0.98, 0.99],
        'quantile_exit_short': [None, 0.9, 0.95, 0.97, 0.98, 0.99],
        'exchange_fee': [0.001, 0.002, 0.003],
        'future': [1]
    }
    
    quantile_model_sweep_results.append(parameter_sweep(
        in_sample[data_part-PADDING:],
        ModelQuantilePredictionsStrategy,
        params,
        params_filter=MODEL_QUANTILE_LOSS_FILTER,
        padding=PADDING,
        interval=INTERVAL,
        sort_by=METRIC))

quantile_model_best_strategies = [[strat for _, strat in results[:TOP_N]] for results in quantile_model_sweep_results]

100%|██████████| 1125/1125 [02:35<00:00,  7.25it/s]


In [103]:
# plot_sweep_results(pd.DataFrame([result for result, _ in quantile_model_sweep_results[0]]), parameters=['quantile_enter_long', 'quantile_exit_long', 'quantile_enter_short', 'quantile_exit_short', 'future'], objective=METRIC)

In [2]:
# Model with gmadl loss
SWEEP_ID = 'filipstefaniuk/wne-masters-thesis-testing/0pro3i5i'
# SWEEP_ID = 'filipstefaniuk/wne-masters-thesis-testing/v3epl3qk'
# train_gmadl_pred_windows = get_sweep_window_predictions(SWEEP_ID, 'train')
valid_gmadl_pred_windows = get_sweep_window_predictions(SWEEP_ID, 'valid')
test_gmadl_pred_windows = get_sweep_window_predictions(SWEEP_ID, 'test')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  


In [13]:
MODEL_GMADL_LOSS_FILTER = lambda p: (
    ((p['enter_long'] is not None and (p['enter_short'] is not None or p['exit_long'] is not None))
    or (p['enter_short'] is not None and (p['exit_short'] is not None or p['enter_long'] is not None)))
    and (p['enter_short'] is None or p['exit_long'] is None or (p['exit_long'] > p['enter_short']))
    and (p['enter_long'] is None or p['exit_short'] is None or (p['exit_short'] < p['enter_long'])))

gmadl_model_sweep_results = []
for (in_sample, _), valid_preds, test_preds in zip(data_windows, valid_gmadl_pred_windows, test_gmadl_pred_windows):
    data_part = int((1 - VALID_PART) * len(in_sample))
    params={
        'predictions': [get_predictions_dataframe(valid_preds, test_preds)],
        'enter_long': [None, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007],
        'exit_long': [None, -0.001, -0.002, -0.003, -0.004, -0.005, -0.006, -0.007],
        'enter_short': [None, -0.001, -0.002, -0.003, -0.004, -0.005, -0.006, -0.007],
        'exit_short': [None, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007],
    }
    
    gmadl_model_sweep_results.append(parameter_sweep(
        in_sample[data_part-PADDING:],
        ModelGmadlPredictionsStrategy,
        params,
        params_filter=MODEL_GMADL_LOSS_FILTER,
        padding=PADDING,
        interval=INTERVAL,
        sort_by=METRIC))
    

gmadl_model_best_strategies = [[strat for _, strat in results[:TOP_N]] for results in gmadl_model_sweep_results]

100%|██████████| 1176/1176 [00:55<00:00, 21.03it/s]


In [15]:
# Persist best strategies, so that they don't have to be recomputed every time
best_strategies = {
    'buy_and_hold': buyandhold_best_strategies,
    'macd_strategies': macd_best_strategies,
    'rsi_strategies': rsi_best_strategies,
    'rmse_model': rmse_model_best_strategies,
    'quantile_model': quantile_model_best_strategies,
    'gmadl_model': gmadl_model_best_strategies
}

with open('cache/5min-best-strategies-v2.pkl', 'wb') as outp:
    pickle.dump(best_strategies, outp, pickle.HIGHEST_PROTOCOL)

## Visualizations & Tables

In [4]:
with open('cache/5min-best-strategies-v2.pkl', 'rb') as inpt:
    best_strategies = pickle.load(inpt)

#### Tables with parameters

In [53]:
# Best hparams for MACD strategy
table_macd_params = Texttable()
table_macd_params.set_deco(Texttable.HEADER)
table_macd_params.set_cols_align(["l", "c", "c", "c", "c"])
table_macd_params.header([
    "\\textbf{Window}",
    "\\textbf{Fast Window Size}",
    "\\textbf{Slow Window Size}",
    "\\textbf{Signal Window Size}",
    "\\textbf{Short sell}"
])

for i, macd_strategy in enumerate(best_strategies['macd_strategies']):
    macd_strategy_info = macd_strategy[0].info()
    table_macd_params.add_row([
        f"Window {i+1}",
        macd_strategy_info['fast_window_size'],
        macd_strategy_info['slow_window_size'],
        macd_strategy_info['signal_window_size'],
        macd_strategy_info['short_sell']
    ])
print(latextable.draw_latex(table_macd_params))

\begin{table}
	\begin{center}
		\begin{tabular}{lcccc}
			\textbf{Window} & \textbf{Fast Window Size} & \textbf{Slow Window Size} & \textbf{Signal Window Size} & \textbf{Short sell} \\
			\hline
			Window 1 & 8 & 2584 & 987 & True \\
			Window 2 & 377 & 610 & 610 & True \\
			Window 3 & 987 & 2584 & 987 & True \\
			Window 4 & 610 & 2584 & 987 & True \\
			Window 5 & 987 & 1597 & 987 & False \\
			Window 6 & 1597 & 2584 & 377 & False \\
		\end{tabular}
	\end{center}
\end{table}


In [55]:
table_rsi_params = Texttable()
table_rsi_params.set_deco(Texttable.HEADER)
table_rsi_params.set_cols_align(["l", "c","c", "c", "c", "c"])
table_rsi_params.header([
    "\\textbf{Window}",
    "\\textbf{\\textit{window}}",
    "\\textbf{\\textit{enter long}}",
    "\\textbf{\\textit{exit long}}",
    "\\textbf{\\textit{enter short}}",
    "\\textbf{\\textit{exit short}}"
])

for i, rsi_strategy in enumerate(best_strategies['rsi_strategies']):
    rsi_strategy_info = rsi_strategy[0].info()
    table_rsi_params.add_row([
        f"W{i+1}-5min",
        rsi_strategy_info['window_size'] or '-',
        rsi_strategy_info['enter_long'] or '-',
        rsi_strategy_info['exit_long'] or '-',
        rsi_strategy_info['enter_short'] or '-',
        rsi_strategy_info['exit_short'] or '-'
    ])
print(latextable.draw_latex(table_rsi_params))

\begin{table}
	\begin{center}
		\begin{tabular}{lccccc}
			\textbf{Window} & \textbf{\textit{window}} & \textbf{\textit{enter long}} & \textbf{\textit{exit long}} & \textbf{\textit{enter short}} & \textbf{\textit{exit short}} \\
			\hline
			W1-5min & 21 & 80 & - & 25 & - \\
			W2-5min & 34 & 75 & - & 25 & - \\
			W3-5min & 5 & 95 & - & 15 & - \\
			W4-5min & 8 & 90 & 5 & - & 85 \\
			W5-5min & 13 & 90 & 20 & - & 85 \\
			W6-5min & 21 & 85 & 15 & - & - \\
		\end{tabular}
	\end{center}
\end{table}


In [5]:
table_rmse_params = Texttable()
table_rmse_params.set_deco(Texttable.HEADER)
table_rmse_params.set_cols_align(["l", "c","c", "c", "c"])
table_rmse_params.header([
    "\\textbf{Window}",
    "\\textbf{\\textit{enter long}}",
    "\\textbf{\\textit{exit Long}}",
    "\\textbf{\\textit{enter Short}}",
    "\\textbf{\\textit{exit Short}}",
])

for i, rmse_strategy in enumerate(best_strategies['rmse_model']):
    rmse_strategy_info = rmse_strategy[0].info()
    table_rmse_params.add_row([
        f"W{i+1}-{INTERVAL}",
        rmse_strategy_info['enter_long'] or '-',
        rmse_strategy_info['exit_long'] or '-',
        rmse_strategy_info['enter_short'] or '-',
        rmse_strategy_info['exit_short'] or '-'
    ])
print(latextable.draw_latex(table_rmse_params))

\begin{table}
	\begin{center}
		\begin{tabular}{lcccc}
			\textbf{Window} & \textbf{\textit{enter long}} & \textbf{\textit{exit Long}} & \textbf{\textit{enter Short}} & \textbf{\textit{exit Short}} \\
			\hline
			W1-5min & 0.002 & - & -0.001 & 0.001 \\
			W2-5min & - & - & -0.001 & 0.001 \\
			W3-5min & - & - & -0.001 & 0.001 \\
			W4-5min & - & - & -0.001 & 0.001 \\
			W5-5min & - & - & -0.001 & 0.001 \\
			W6-5min & - & - & -0.001 & 0.001 \\
		\end{tabular}
	\end{center}
\end{table}


In [56]:
table_quantile_params = Texttable()
table_quantile_params.set_deco(Texttable.HEADER)
table_quantile_params.set_cols_align(["l", "c","c", "c", "c", "c"])
table_quantile_params.header([
    "\\textbf{Window}",
    "\\textbf{\\textit{enter long}}",
    "\\textbf{\\textit{exit Long}}",
    "\\textbf{\\textit{enter Short}}",
    "\\textbf{\\textit{exit Short}}",
    "\\textbf{\\textit{threshold}}"
])

for i, quantile_strategy in enumerate(best_strategies['quantile_model']):
    quantile_strategy_info = quantile_strategy[0].info()
    table_quantile_params.add_row([
        f"W{i+1}-{INTERVAL}",
        quantile_strategy_info['quantile_enter_long'] or '-',
        quantile_strategy_info['quantile_exit_long'] or '-',
        quantile_strategy_info['quantile_enter_short'] or '-',
        quantile_strategy_info['quantile_exit_short'] or '-',
        quantile_strategy_info['exchange_fee']
    ])
print(latextable.draw_latex(table_quantile_params))

\begin{table}
	\begin{center}
		\begin{tabular}{lccccc}
			\textbf{Window} & \textbf{\textit{enter long}} & \textbf{\textit{exit Long}} & \textbf{\textit{enter Short}} & \textbf{\textit{exit Short}} & \textbf{\textit{threshold}} \\
			\hline
			W1-5min & - & 0.970 & 0.980 & 0.970 & 0.003 \\
			W2-5min & 0.970 & - & 0.950 & - & 0.003 \\
			W3-5min & 0.990 & - & 0.950 & - & 0.002 \\
			W4-5min & 0.970 & 0.990 & - & 0.900 & 0.003 \\
			W5-5min & 0.900 & 0.900 & - & - & 0.002 \\
			W6-5min & 0.950 & 0.950 & - & - & 0.003 \\
		\end{tabular}
	\end{center}
\end{table}


In [57]:
table_gmadl_params = Texttable()
table_gmadl_params.set_deco(Texttable.HEADER)
table_gmadl_params.set_cols_align(["l", "c","c", "c", "c"])
table_gmadl_params.header([
    "\\textbf{Window}",
    "\\textbf{\\textit{enter long}}",
    "\\textbf{\\textit{exit Long}}",
    "\\textbf{\\textit{enter Short}}",
    "\\textbf{\\textit{exit Short}}",
])

for i, gmadl_strategy in enumerate(best_strategies['gmadl_model']):
    gmadl_strategy_info = gmadl_strategy[0].info()
    table_gmadl_params.add_row([
        f"W{i+1}-{INTERVAL}",
        gmadl_strategy_info['enter_long'] or '-',
        gmadl_strategy_info['exit_long'] or '-',
        gmadl_strategy_info['enter_short'] or '-',
        gmadl_strategy_info['exit_short'] or '-'
    ])
print(latextable.draw_latex(table_gmadl_params))

\begin{table}
	\begin{center}
		\begin{tabular}{lcccc}
			\textbf{Window} & \textbf{\textit{enter long}} & \textbf{\textit{exit Long}} & \textbf{\textit{enter Short}} & \textbf{\textit{exit Short}} \\
			\hline
			W1-5min & 0.004 & - & -0.005 & - \\
			W2-5min & 0.002 & - & -0.001 & - \\
			W3-5min & - & - & -0.006 & 0.003 \\
			W4-5min & 0.002 & - & -0.005 & - \\
			W5-5min & 0.002 & - & -0.003 & - \\
			W6-5min & 0.001 & - & -0.007 & - \\
		\end{tabular}
	\end{center}
\end{table}


#### Evaluation on the windows

In [6]:
def results_plot(idx, result_buyandhold, result_macd, result_rsi, result_rmse_model, result_quantile_model, result_gmadl_model, width=850, height=500, notitle=False, v_lines=None):

    fig = go.Figure([
        go.Scatter(y=result_buyandhold['portfolio_value'], x=result_buyandhold['time'], name="Buy and Hold"),
        go.Scatter(y=result_macd['portfolio_value'], x=result_macd['time'], name="MACD Strategy"),
        go.Scatter(y=result_rsi['portfolio_value'], x=result_rsi['time'], name="RSI Strategy"),
        go.Scatter(y=result_rmse_model['portfolio_value'], x=result_rmse_model['time'], name='RMSE Informer Strategy'),
        go.Scatter(y=result_quantile_model['portfolio_value'], x=result_quantile_model['time'], name='Quantile Informer Strategy'),
        go.Scatter(y=result_gmadl_model['portfolio_value'], x=result_gmadl_model['time'], name='GMADL Informer Strategy')
        ])
    
    if v_lines:
        for v_line in v_lines:
            fig.add_shape(
                go.layout.Shape(type="line",
                                yref="paper",
                                xref="x",
                                x0=v_line,
                                x1=v_line,
                                y0=0,
                                y1=1,
                                line=dict(dash='dash', color='rgb(140,140,140)')))
    fig.update_layout(
        title={
        'text': f"W{idx}-{INTERVAL}",
        'y':0.97,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'} if not notitle else None,
        yaxis_title="Portfolio Value",
        xaxis_title="Date",
            font=dict(
            # family="Courier New, monospace",
            size=14,
        ),
        autosize=False,
        width=width,
        height=height,
        margin=dict(l=20, r=20, t=20 if notitle else 110, b=20),
        plot_bgcolor='white',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="left",
            x=0.02
        )
    )
    fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
    )
    fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
    )
    fig.write_image(f"images/eval-w{idx}-{INTERVAL}.png")
    fig.show()
    
def results_table(result_buyandhold, result_macd, result_rsi, result_rmse_model, result_quantile_model, result_gmadl_model):
    table_eval_windows = Texttable()
    table_eval_windows.set_deco(Texttable.HEADER)
    table_eval_windows.set_cols_align(["l", "c","c", "c", "c", "c", "c", "c", "c", "c"])
    table_eval_windows.set_precision(3)

    table_eval_windows.header([
        "\\textbf{Strategy}",
        "\\textbf{VAL}",
        "\\textbf{ARC}",
        "\\textbf{ASD}",
        "\\textbf{IR*}",
        "\\textbf{MD}",
        "\\textbf{IR**}",
        "\\textbf{N}",
        "\\textbf{LONG}",
        "\\textbf{SHORT}",
    ])

    strategy_name_result = [
        ('Buy and Hold', result_buyandhold),
        ('MACD Strategy', result_macd),
        ('RSI Strategy', result_rsi),
        ('RMSE Informer', result_rmse_model),
        ('Quantile Informer', result_quantile_model),
        ('GMADL Informer', result_gmadl_model)
    ]
    for strategy_name, result in strategy_name_result:
        table_eval_windows.add_row([
            strategy_name,
            result['value'],
            f"{result['arc']*100:.2f}\%",
            f"{result['asd']*100:.2f}\%",
            result['ir'],
            f"{result['md']*100:.2f}\%",
            result['mod_ir'],
            result['n_trades'],
            f"{result['long_pos']*100:.2f}\%",
            f"{result['short_pos']*100:.2f}\%",
        ])
    print(latextable.draw_latex(table_eval_windows))



In [11]:
for i, (in_sample, out_of_sample) in enumerate(data_windows):
    padded_window = pd.concat([in_sample.iloc[-PADDING:], out_of_sample])
    result_buyandhold = evaluate_strategy(padded_window, best_strategies['buy_and_hold'][i], padding=PADDING, interval=INTERVAL)
    result_macd = evaluate_strategy(padded_window, [s[0] for s in best_strategies['macd_strategies']][i], padding=PADDING, interval=INTERVAL)
    result_rsi = evaluate_strategy(padded_window, [s[0] for s in best_strategies['rsi_strategies']][i], padding=PADDING, interval=INTERVAL)
    result_rmse_model = evaluate_strategy(padded_window, [s[0] for s in best_strategies['rmse_model']][i], padding=PADDING, interval=INTERVAL)
    result_quantile_model = evaluate_strategy(padded_window, [s[0] for s in best_strategies['quantile_model']][i], padding=PADDING, interval=INTERVAL)
    result_gmadl_model = evaluate_strategy(padded_window, [s[0] for s in best_strategies['gmadl_model']][i], padding=PADDING, interval=INTERVAL)

    results_table(result_buyandhold, result_macd, result_rsi, result_rmse_model, result_quantile_model, result_gmadl_model)
    # results_plot(i+1, result_buyandhold, result_macd, result_rsi, result_rmse_model, result_quantile_model, result_gmadl_model)
        


\begin{table}
	\begin{center}
		\begin{tabular}{lccccccccc}
			\textbf{Strategy} & \textbf{VAL} & \textbf{ARC} & \textbf{ASD} & \textbf{IR*} & \textbf{MD} & \textbf{IR**} & \textbf{N} & \textbf{LONG} & \textbf{SHORT} \\
			\hline
			Buy and Hold & 0.930 & -13.76\% & 70.24\% & -0.196 & 51.87\% & -0.052 & 2 & 100.00\% & 0.00\% \\
			MACD Strategy & 0.375 & -86.35\% & 70.71\% & -1.221 & 71.90\% & -1.467 & 1542 & 52.05\% & 47.95\% \\
			RSI Strategy & 1.428 & 106.05\% & 70.55\% & 1.503 & 26.24\% & 6.074 & 114 & 26.31\% & 73.69\% \\
			RMSE Informer & 0.965 & -6.93\% & 3.11\% & -2.227 & 3.54\% & -4.361 & 12 & 0.02\% & 0.00\% \\
			Quantile Informer & 1.117 & 25.15\% & 43.69\% & 0.576 & 16.27\% & 0.889 & 365 & 0.00\% & 45.14\% \\
			GMADL Informer & 1.502 & 128.10\% & 70.54\% & 1.816 & 30.70\% & 7.576 & 82 & 41.54\% & 58.46\% \\
		\end{tabular}
	\end{center}
\end{table}
\begin{table}
	\begin{center}
		\begin{tabular}{lccccccccc}
			\textbf{Strategy} & \textbf{VAL} & \textbf{ARC} & \textbf{AS

In [12]:
test_data = pd.concat([data_windows[0][0][-PADDING:]] + [data_window[1] for data_window in data_windows])
buy_and_hold_concat = evaluate_strategy(test_data, BuyAndHoldStrategy(), padding=PADDING)
macd_concat = evaluate_strategy(test_data, ConcatenatedStrategies(len(data_windows[0][1]), [s[0] for s in best_strategies['macd_strategies']], padding=PADDING), padding=PADDING, interval=INTERVAL)
rsi_concat = evaluate_strategy(test_data, ConcatenatedStrategies(len(data_windows[0][1]), [s[0] for s in best_strategies['rsi_strategies']], padding=PADDING), padding=PADDING, interval=INTERVAL)
rmse_model_concat = evaluate_strategy(test_data, ConcatenatedStrategies(len(data_windows[0][1]), [s[0] for s in best_strategies['rmse_model']], padding=PADDING), padding=PADDING, interval=INTERVAL)
quantile_model_concat = evaluate_strategy(test_data, ConcatenatedStrategies(len(data_windows[0][1]), [s[0] for s in best_strategies['quantile_model']], padding=PADDING), padding=PADDING, interval=INTERVAL)
gmadl_model_concat = evaluate_strategy(test_data, ConcatenatedStrategies(len(data_windows[0][1]), [s[0] for s in best_strategies['gmadl_model']], padding=PADDING), padding=PADDING, interval=INTERVAL)

v_lines=[data_window[1]['close_time'].iloc[-1] for data_window in data_windows][:-1]
results_table(buy_and_hold_concat, macd_concat, rsi_concat, rmse_model_concat, quantile_model_concat, gmadl_model_concat)
# results_plot(0, buy_and_hold_concat, macd_concat, rsi_concat, rmse_model_concat, quantile_model_concat, gmadl_model_concat, width=1300, height=500, notitle=True)



\begin{table}
	\begin{center}
		\begin{tabular}{lccccccccc}
			\textbf{Strategy} & \textbf{VAL} & \textbf{ARC} & \textbf{ASD} & \textbf{IR*} & \textbf{MD} & \textbf{IR**} & \textbf{N} & \textbf{LONG} & \textbf{SHORT} \\
			\hline
			Buy and Hold & 1.441 & 13.14\% & 57.74\% & 0.228 & 77.31\% & 0.039 & 2 & 100.00\% & 0.00\% \\
			MACD Strategy & 0.516 & -20.04\% & 54.14\% & -0.370 & 85.77\% & -0.087 & 2535 & 50.39\% & 32.38\% \\
			RSI Strategy & 3.341 & 50.34\% & 50.41\% & 0.999 & 29.99\% & 1.676 & 846 & 28.29\% & 33.47\% \\
			RMSE Informer & 0.643 & -13.88\% & 15.13\% & -0.917 & 44.61\% & -0.285 & 16 & 0.00\% & 9.58\% \\
			Quantile Informer & 0.956 & -1.52\% & 47.91\% & -0.032 & 53.96\% & -0.001 & 3395 & 40.24\% & 27.84\% \\
			GMADL Informer & 9.747 & 115.88\% & 54.44\% & 2.129 & 32.66\% & 7.552 & 846 & 44.80\% & 41.51\% \\
		\end{tabular}
	\end{center}
\end{table}


In [7]:
# No transaction fees
test_data = pd.concat([data_windows[0][0][-PADDING:]] + [data_window[1] for data_window in data_windows])
buy_and_hold_concat = evaluate_strategy(test_data, BuyAndHoldStrategy(), padding=PADDING)
macd_concat = evaluate_strategy(test_data, ConcatenatedStrategies(len(data_windows[0][1]), [s[0] for s in best_strategies['macd_strategies']], padding=PADDING), padding=PADDING, interval=INTERVAL, exchange_fee=0)
rsi_concat = evaluate_strategy(test_data, ConcatenatedStrategies(len(data_windows[0][1]), [s[0] for s in best_strategies['rsi_strategies']], padding=PADDING), padding=PADDING, interval=INTERVAL, exchange_fee=0)
rmse_model_concat = evaluate_strategy(test_data, ConcatenatedStrategies(len(data_windows[0][1]), [s[0] for s in best_strategies['rmse_model']], padding=PADDING), padding=PADDING, interval=INTERVAL, exchange_fee=0)
quantile_model_concat = evaluate_strategy(test_data, ConcatenatedStrategies(len(data_windows[0][1]), [s[0] for s in best_strategies['quantile_model']], padding=PADDING), padding=PADDING, interval=INTERVAL, exchange_fee=0)
gmadl_model_concat = evaluate_strategy(test_data, ConcatenatedStrategies(len(data_windows[0][1]), [s[0] for s in best_strategies['gmadl_model']], padding=PADDING), padding=PADDING, interval=INTERVAL, exchange_fee=0)

v_lines=[data_window[1]['close_time'].iloc[-1] for data_window in data_windows][:-1]
results_table(buy_and_hold_concat, macd_concat, rsi_concat, rmse_model_concat, quantile_model_concat, gmadl_model_concat)
# results_plot(0, buy_and_hold_concat, macd_concat, rsi_concat, rmse_model_concat, quantile_model_concat, gmadl_model_concat, width=1300, height=500, notitle=True)



\begin{table}
	\begin{center}
		\begin{tabular}{lccccccccc}
			\textbf{Strategy} & \textbf{VAL} & \textbf{ARC} & \textbf{ASD} & \textbf{IR*} & \textbf{MD} & \textbf{IR**} & \textbf{N} & \textbf{LONG} & \textbf{SHORT} \\
			\hline
			Buy and Hold & 1.441 & 13.14\% & 57.74\% & 0.228 & 77.31\% & 0.039 & 2 & 100.00\% & 0.00\% \\
			MACD Strategy & 1.918 & 24.63\% & 54.07\% & 0.456 & 67.19\% & 0.167 & 2535 & 50.39\% & 32.38\% \\
			RSI Strategy & 5.154 & 74.05\% & 50.37\% & 1.470 & 26.46\% & 4.113 & 846 & 28.29\% & 33.47\% \\
			RMSE Informer & 0.650 & -13.53\% & 15.13\% & -0.894 & 44.05\% & -0.275 & 16 & 0.00\% & 9.58\% \\
			Quantile Informer & 5.404 & 76.86\% & 47.83\% & 1.607 & 37.34\% & 3.307 & 3395 & 40.24\% & 27.84\% \\
			GMADL Informer & 14.946 & 149.43\% & 54.42\% & 2.746 & 31.57\% & 12.994 & 846 & 44.80\% & 41.51\% \\
		\end{tabular}
	\end{center}
\end{table}
